# Setting up initial

In [12]:
# import pandas as pd
# import requests
# import os
# import re
# import time
# from time import time
# from IPython.display import display, clear_output
# from tqdm import tqdm

import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import display, clear_output

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 200)



In [13]:
# Read csv file from google drive
url='https://drive.google.com/file/d/1RyMAQvNOl8l2KsWSG6yFPXKN3WbHv25N/view?usp=sharing'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df_sf_scan = pd.read_csv(dwn_url)

print(len(df_sf_scan))

df_sf_scan = df_sf_scan[df_sf_scan['Status Code'] == 200]
df_crime_in = df_sf_scan['Address'][df_sf_scan['Address'].str.contains('/crime/in/')]

print(len(df_sf_scan))
print(len(df_crime_in))

183240
183204
13480


In [ ]:
start_elapsed_time = time()

# Function to append data to the CSV files
def append_data_to_csv(df, file_name):
    with open(file_name, 'a', encoding='utf-8', newline='') as f:
        df.to_csv(f, header=f.tell() == 0, index=False)

# Initialize an empty list to store the scraped DF's
scraped_data_1 = []
scraped_data_2 = []
start_page = 1  # Starting page number
end_page = len(df_crime_in)
#end_page = 10   # Ending page number (not inclusive)
count = start_page
count_status_code_200_yes = 0
count_status_code_200_no = 0
table_number_1 = 2
table_number_2 = 3
num_pages_append = 100

# Create empty DF's
df = pd.DataFrame()
df_timing = pd.DataFrame(columns=['url', 'start_time', 'end_time', 'page_count'])

# Iterate through the URLs from the desired range and scrape the specific table in the HTML file
for index, urls in enumerate(df_crime_in[start_page-1:end_page], start=1):
    start_time = time()

    response = requests.get(urls)
    if response.status_code == 200:
        html_content = response.text
        read_html_tables = pd.read_html(html_content)
        soup = BeautifulSoup(html_content, 'html.parser')

        # Check if the table_number is within the valid range
        if table_number_1 >= 0 and table_number_1 < len(read_html_tables) and table_number_2 >= 0 and table_number_2 < len(read_html_tables):
            df_table_1 = read_html_tables[table_number_1]
            df_table_2 = read_html_tables[table_number_2]

            ## Concat 2 tables from the html to 1 df_table
            #df_table = pd.concat([df_table_1, df_table_2], ignore_index=True)

            # Add the 'URL' column containing the current URL value to the DF
            df_table_1['URL'] = urls
            df_table_2['URL'] = urls

            # Extract the country and city from the breadcrumbs element
            column_1 = soup.find('span', itemprop='name').text
            page_country = soup.find_all('span', itemprop='name')[1].text
            page_city = soup.find_all('span', itemprop='name')[2].text

            df_table_1['column_1'] = column_1
            df_table_1['Country'] = page_country
            df_table_1['City'] = page_city
            df_table_2['column_1'] = column_1
            df_table_2['Country'] = page_country
            df_table_2['City'] = page_city

            scraped_data_1.append(df_table_1)
            scraped_data_2.append(df_table_2)
            end_time = time()
            clear_output(wait=True)
            count += 1
            count_status_code_200_yes += 1
            display(f'Page {count}/{end_page} | {urls} | {round(end_time - start_time, 2)} sec')

            # Add data to df_timing to keep track of numbeo response times
            df_timing = pd.concat([df_timing, pd.DataFrame({'url': [urls], 'start_time': [start_time], 'end_time': [end_time], 'page_count': [count]})], ignore_index=True)

            # Append data to CSV files every num_pages_append pages or on the last page
            if count % num_pages_append == 0 or count == end_page:
                if scraped_data_1:
                    df_raw_1 = pd.concat(scraped_data_1, ignore_index=True)
                    df_1 = df_raw_1

                    # Save the DF's to CSV files
                    append_data_to_csv(df_1, 'df_output_1.csv')
                    append_data_to_csv(df_timing, 'df_timing_output.csv')

                    # Clear the scraped_data list and reinitialize df and df_timing
                    scraped_data_1 = []
                    df_1 = pd.DataFrame()
                    df_timing = pd.DataFrame()


                if scraped_data_2:
                    df_raw_2 = pd.concat(scraped_data_2, ignore_index=True)
                    df_2 = df_raw_2

                    # Save the DF's to CSV files
                    append_data_to_csv(df_2, 'df_output_2.csv')

                    # Clear the scraped_data list and reinitialize df and df_timing
                    scraped_data_2 = []
                    df_2 = pd.DataFrame()

                    clear_output(wait=True)
                    print('\033[32mData appended to CSV files!\033[0m')  # Green color
        else:
            clear_output(wait=True)
            count += 1
            count_status_code_200_no += 1
            display(f'Page {count}/{end_page-1} | {urls} | Oops... no table')
            continue
    else:
        clear_output(wait=True)
        count += 1
        count_status_code_200_no += 1
        display(f'Page {count}/{end_page-1} | {urls} | Error: Unable to fetch data')
        continue

# Calculate time_elapsed in seconds
time_elapsed = time() - start_elapsed_time
# Format time_elapsed as hh:mm:ss
formatted_time_elapsed = '{:.0f}:{:.0f}:{:.0f}'.format(time_elapsed // 3600, (time_elapsed % 3600) // 60, time_elapsed % 60)

# Final message after scraping all pages
clear_output(wait=True)
print(f'\033[32mDone!\033[0m | Number of scrapped URLs: {count_status_code_200_yes} / {end_page}')  # Green color
print(formatted_time_elapsed)

'Page 6484/13479 | https://www.numbeo.com/crime/in/Mahebourg-Mauritius | Oops... no table'